In [160]:
from bs4 import BeautifulSoup
from datetime import time
from datetime import datetime
from datetime import timedelta
from collections import Counter
from tqdm import tqdm
import json
import requests
import csv
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import concurrent.futures
import io
import re
import chardet

In [161]:
Women_Leaderboard = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Scraped_Data/WomenOpenLeaderboard.csv')
Women_Leaderboard.head()

,CompetitorID,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height,Age,Weight,Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,2080362,Mirjam von Rohr,Switzerland,Europe,CrossFit Ouf,F,NaN,22,NaN,1,1,37,2
1,1976672,Grace Walton,Australia,Oceania,NaN,F,164 cm,23,70 kg,2,8,3,33
2,1239579,Anikha Greer,Canada,North America East,Peak 360 CrossFit,F,154 cm,21,NaN,3,42,27,3
3,132258,Arielle Loewen,United States,North America West,NaN,F,63 in,30,150 lb,4,21,4,52
4,239148,Carolyne Prevost,Canada,North America East,CrossFit Colosseum,F,63 in,34,145 lb,5,15,59,17


In [162]:
# Function to convert kilograms to pounds and round to the nearest pound
def kg_to_lb(weight):
    if isinstance(weight, str):
        if 'kg' in weight:
            return round(float(weight.split()[0]) * 2.20462)
        else:
            return round(float(weight.split()[0]))
    elif isinstance(weight, float) and np.isnan(weight):
        return np.nan  # Return NaN if the weight is NaN
    else:
        return round(weight * 2.20462)  # If the weight is already in lbs, just return it

# Apply the conversion function to the Weight column
Women_Leaderboard['Weight'] = Women_Leaderboard['Weight'].apply(kg_to_lb)

# Print the updated DataFrame
Women_Leaderboard.head()

,CompetitorID,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height,Age,Weight,Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,2080362,Mirjam von Rohr,Switzerland,Europe,CrossFit Ouf,F,NaN,22,NaN,1,1,37,2
1,1976672,Grace Walton,Australia,Oceania,NaN,F,164 cm,23,154.0,2,8,3,33
2,1239579,Anikha Greer,Canada,North America East,Peak 360 CrossFit,F,154 cm,21,NaN,3,42,27,3
3,132258,Arielle Loewen,United States,North America West,NaN,F,63 in,30,150.0,4,21,4,52
4,239148,Carolyne Prevost,Canada,North America East,CrossFit Colosseum,F,63 in,34,145.0,5,15,59,17


In [163]:
# Rename the 'Weight' column to 'Weight (lbs)'
Women_Leaderboard.rename(columns={'Weight': 'Weight (lbs)'}, inplace=True)

Women_Leaderboard.head()

,CompetitorID,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height,Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,2080362,Mirjam von Rohr,Switzerland,Europe,CrossFit Ouf,F,NaN,22,NaN,1,1,37,2
1,1976672,Grace Walton,Australia,Oceania,NaN,F,164 cm,23,154.0,2,8,3,33
2,1239579,Anikha Greer,Canada,North America East,Peak 360 CrossFit,F,154 cm,21,NaN,3,42,27,3
3,132258,Arielle Loewen,United States,North America West,NaN,F,63 in,30,150.0,4,21,4,52
4,239148,Carolyne Prevost,Canada,North America East,CrossFit Colosseum,F,63 in,34,145.0,5,15,59,17


In [164]:
# Function to convert centimeters to inches
def cm_to_inches(height):
    if isinstance(height, str):
        if 'cm' in height:
            return round(float(height.split()[0]) * 0.393701)
        else:
            return int(height.split()[0])
    elif isinstance(height, float) and np.isnan(height):
        return np.nan  # Return NaN if the height is NaN
    else:
        return height  # If the height is already in inches, just return it

# Apply the conversion function to the Height column
Women_Leaderboard['Height'] = Women_Leaderboard['Height'].apply(cm_to_inches)

# Print the updated DataFrame
Women_Leaderboard.head()

,CompetitorID,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height,Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,2080362,Mirjam von Rohr,Switzerland,Europe,CrossFit Ouf,F,NaN,22,NaN,1,1,37,2
1,1976672,Grace Walton,Australia,Oceania,NaN,F,65.0,23,154.0,2,8,3,33
2,1239579,Anikha Greer,Canada,North America East,Peak 360 CrossFit,F,61.0,21,NaN,3,42,27,3
3,132258,Arielle Loewen,United States,North America West,NaN,F,63.0,30,150.0,4,21,4,52
4,239148,Carolyne Prevost,Canada,North America East,CrossFit Colosseum,F,63.0,34,145.0,5,15,59,17


In [165]:
# Rename the 'Height' column to 'Height (in))'
Women_Leaderboard.rename(columns={'Height': 'Height (in)'}, inplace=True)
Women_Leaderboard.head()

,CompetitorID,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,2080362,Mirjam von Rohr,Switzerland,Europe,CrossFit Ouf,F,NaN,22,NaN,1,1,37,2
1,1976672,Grace Walton,Australia,Oceania,NaN,F,65.0,23,154.0,2,8,3,33
2,1239579,Anikha Greer,Canada,North America East,Peak 360 CrossFit,F,61.0,21,NaN,3,42,27,3
3,132258,Arielle Loewen,United States,North America West,NaN,F,63.0,30,150.0,4,21,4,52
4,239148,Carolyne Prevost,Canada,North America East,CrossFit Colosseum,F,63.0,34,145.0,5,15,59,17


In [166]:
# Get unique values and sort them from highest to lowest
New_Height_sorted_Women = sorted(Women_Leaderboard['Height (in)'].unique(), reverse=True)
print(New_Height_sorted_Women)

[nan, 25801.0, 6181.0, 6024.0, 2040.0, 1956.0, 709.0, 669.0, 665.0, 661.0, 654.0, 650.0, 642.0, 638.0, 630.0, 600.0, 510.0, 504.0, 444.0, 420.0, 415.0, 411.0, 220.0, 214.0, 203.0, 189.0, 180.0, 178.0, 177.0, 175.0, 174.0, 173.0, 172.0, 171.0, 170.0, 169.0, 168.0, 167.0, 166.0, 165.0, 164.0, 163.0, 162.0, 161.0, 160.0, 159.0, 158.0, 157.0, 156.0, 154.0, 153.0, 150.0, 128.0, 118.0, 114.0, 110.0, 96.0, 84.0, 82.0, 81.0, 80.0, 79.0, 77.0, 76.0, 75.0, 74.0, 73.0, 72.0, 71.0, 70.0, 69.0, 68.0, 67.0, 66.0, 65.0, 64.0, 63.0, 62.0, 61.0, 60.0, 59.0, 58.0, 57.0, 56.0, 55.0, 54.0, 53.0, 52.0, 48.0, 47.0, 46.0, 42.0, 40.0, 29.0, 28.0, 27.0, 26.0, 25.0, 24.0, 23.0, 21.0, 18.0, 17.0, 16.0, 14.0, 12.0, 6.0, 5.0, 4.0, 2.0, 1.0, 0.0]


In [167]:
# Specify the correct encoding
encoding = 'latin1'  # or 'ISO-8859-1', etc. - choose the one that works for your file

# Function to remove specific lines from a file
def remove_lines(file_path, lines_to_remove):
    with open(file_path, 'r', encoding=encoding) as file:
        lines = file.readlines()
    
    # Remove the specified lines
    for line_number in sorted(lines_to_remove, reverse=True):
        del lines[line_number]

    with open(file_path, 'w', encoding=encoding) as file:
        file.writelines(lines)

# List of line numbers to remove
lines_to_remove = [14450, 42076, 49668, 84718, 87927, 88153, 92057]  # Adjust line numbers as needed

# Remove specified lines from the CSV file
remove_lines('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Scraped_Data/WomenPastOpen.csv', lines_to_remove)

# Read the modified CSV file into a DataFrame
Women_Past = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Scraped_Data/WomenPastOpen.csv', encoding=encoding)

# Set pandas display options to show all columns without truncation
pd.set_option('display.max_columns', None)

# Display the DataFrame
Women_Past.head()

,Athlete ID,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
0,1775,6060thWomen,1556thWomen,1921stWomen,1248thWomen,978thWomen,786thWomen,1203rdWomen,7011thWomen,--,4704thWomen,4074thWomen,--,7217thWomen,--
1,1755,70388thWomen,--,--,--,--,14769thWomen,20353rdWomen,14188thWomen,8850thWomen,7446thWomen,8116thWomen,12165thWomen,9301stWomen,--
2,1802,37421stWomen,14274thWomen,82750thWomen,17629thWomen,--,8785thWomen,5435thWomen,8117thWomen,7161stWomen,1794thWomen,5121stWomen,1126thWomen,1289thWomen,1185thWomen
3,1751,35925thWomen,17813thWomen,--,--,--,32938thWomen,53678thWomen,8417thWomen,12282ndWomen,Women,--,84thWomen,712thWomen,418thWomen
4,1659,3944thWomen,--,1781stWomen,--,--,--,1628thWomen,864thWomen,510thWomen,582ndWomen,5986thWomen,1042ndWomen,2213thWomen,--


In [168]:
# Replace '--' with NaN
Women_Past.replace('--', np.nan, inplace=True)

# Print the updated DataFrame
Women_Past.head()

,Athlete ID,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
0,1775,6060thWomen,1556thWomen,1921stWomen,1248thWomen,978thWomen,786thWomen,1203rdWomen,7011thWomen,NaN,4704thWomen,4074thWomen,NaN,7217thWomen,NaN
1,1755,70388thWomen,NaN,NaN,NaN,NaN,14769thWomen,20353rdWomen,14188thWomen,8850thWomen,7446thWomen,8116thWomen,12165thWomen,9301stWomen,NaN
2,1802,37421stWomen,14274thWomen,82750thWomen,17629thWomen,NaN,8785thWomen,5435thWomen,8117thWomen,7161stWomen,1794thWomen,5121stWomen,1126thWomen,1289thWomen,1185thWomen
3,1751,35925thWomen,17813thWomen,NaN,NaN,NaN,32938thWomen,53678thWomen,8417thWomen,12282ndWomen,Women,NaN,84thWomen,712thWomen,418thWomen
4,1659,3944thWomen,NaN,1781stWomen,NaN,NaN,NaN,1628thWomen,864thWomen,510thWomen,582ndWomen,5986thWomen,1042ndWomen,2213thWomen,NaN


In [169]:
#This gets rid of all the letters 
def extract_numbers(string):
    if pd.isnull(string):  # If the value is NaN, return NaN
        return np.nan
    elif isinstance(string, float):  # If the value is float, return NaN
        return np.nan
    else:
        numbers = re.findall(r'\d+', string)  # Extract numbers using regex
        if numbers:  # If numbers are found
            return int(numbers[0])  # Convert the first number found to an integer
        else:  # If no numbers found
            return np.nan  # Return NaN

# Apply the function to all columns except 'Athlete ID'
Women_Past[Women_Past.columns.difference(['Athlete ID'])] = Women_Past[Women_Past.columns.difference(['Athlete ID'])].applymap(extract_numbers)

# Print the updated DataFrame
Women_Past.head()

,Athlete ID,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
0,1775,6060.0,1556.0,1921.0,1248.0,978.0,786.0,1203.0,7011.0,NaN,4704.0,4074.0,NaN,7217.0,NaN
1,1755,70388.0,NaN,NaN,NaN,NaN,14769.0,20353.0,14188.0,8850.0,7446.0,8116.0,12165.0,9301.0,NaN
2,1802,37421.0,14274.0,82750.0,17629.0,NaN,8785.0,5435.0,8117.0,7161.0,1794.0,5121.0,1126.0,1289.0,1185.0
3,1751,35925.0,17813.0,NaN,NaN,NaN,32938.0,53678.0,8417.0,12282.0,NaN,NaN,84.0,712.0,418.0
4,1659,3944.0,NaN,1781.0,NaN,NaN,NaN,1628.0,864.0,510.0,582.0,5986.0,1042.0,2213.0,NaN


In [170]:
# Define a function to add the word "Rank" after all the years in the columns
def add_rank_suffix(col):
    if col.isdigit():
        return col + ' Rank'
    else:
        return col

# Rename the columns except 'Athlete ID'
Women_Past.rename(columns={col: add_rank_suffix(col) for col in Women_Past.columns if col != 'Athlete ID'}, inplace=True)

Women_Past.head()

,Athlete ID,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank
0,1775,6060.0,1556.0,1921.0,1248.0,978.0,786.0,1203.0,7011.0,NaN,4704.0,4074.0,NaN,7217.0,NaN
1,1755,70388.0,NaN,NaN,NaN,NaN,14769.0,20353.0,14188.0,8850.0,7446.0,8116.0,12165.0,9301.0,NaN
2,1802,37421.0,14274.0,82750.0,17629.0,NaN,8785.0,5435.0,8117.0,7161.0,1794.0,5121.0,1126.0,1289.0,1185.0
3,1751,35925.0,17813.0,NaN,NaN,NaN,32938.0,53678.0,8417.0,12282.0,NaN,NaN,84.0,712.0,418.0
4,1659,3944.0,NaN,1781.0,NaN,NaN,NaN,1628.0,864.0,510.0,582.0,5986.0,1042.0,2213.0,NaN


In [171]:
# Specify the correct encoding
encoding = 'latin1'  # or 'ISO-8859-1', etc. - choose the one that works for your file

# Function to remove specific lines from a file
def remove_lines(file_path, lines_to_remove):
    with open(file_path, 'r', encoding=encoding) as file:
        lines = file.readlines()
    
    # Remove the specified lines
    for line_number in sorted(lines_to_remove, reverse=True):
        del lines[line_number]

    with open(file_path, 'w', encoding=encoding) as file:
        file.writelines(lines)

# List of line numbers to remove
lines_to_remove = [6335, 19690, 96172, 97074, 102157, 119426]  # Adjust line numbers as needed

# Remove specified lines from the CSV file
remove_lines('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Scraped_Data/WomenBenchmarks.csv', lines_to_remove)

# Read the modified CSV file into a DataFrame
Women_Benchmarks = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Scraped_Data/WomenBenchmarks.csv', encoding=encoding)

# Set pandas display options to show all columns without truncation
pd.set_option('display.max_columns', None)

# Display the DataFrame
Women_Benchmarks.head()

,Athlete ID,Back Squat,Chad1000x,Clean and Jerk,Deadlift,Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Max Pull-Ups,Run 5k,Snatch,Sprint 400m
0,1755,245 lb,--,165 lb,265 lb,293,--,4:29,2:25,11:09,--,--,25:08,120 lb,1:20
1,1775,303 lb,--,231 lb,345 lb,--,--,2:37,1:54,10:22,--,--,--,190 lb,--
2,1659,300 lb,--,235 lb,350 lb,--,--,2:15,--,--,--,--,23:51,190 lb,1:00
3,1802,250 lb,--,200 lb,330 lb,320,--,3:30,--,--,--,--,--,160 lb,1:23
4,1751,120 kg,--,97 kg,166 kg,--,22:54,3:39,1:52,9:11,--,--,--,73 kg,--


In [172]:
del Women_Benchmarks['Max Pull-Ups']
del Women_Benchmarks['Chad1000x']

In [173]:
# Replace '--' with NaN
Women_Benchmarks.replace('--', np.nan, inplace=True)

# Print the updated DataFrame
Women_Benchmarks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143517 entries, 0 to 143516
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Athlete ID      143497 non-null  object
 1   Back Squat      27356 non-null   object
 2   Clean and Jerk  24739 non-null   object
 3   Deadlift        28672 non-null   object
 4   Fight Gone Bad  4481 non-null    object
 5   Filthy 50       2315 non-null    object
 6   Fran            8774 non-null    object
 7   Grace           8149 non-null    object
 8   Helen           5920 non-null    object
 9   L1 Benchmark    199 non-null     object
 10  Run 5k          9415 non-null    object
 11  Snatch          22663 non-null   object
 12  Sprint 400m     3578 non-null    object
dtypes: object(13)
memory usage: 14.2+ MB


In [174]:
# Define a function to remove 'lb' and convert to pounds
def clean_and_convert(weight):
    try:
        if 'kg' in str(weight):
            return pd.to_numeric(weight.split()[0]) * 2.20462
        elif 'lb' in str(weight):
            return pd.to_numeric(weight.split()[0])
        else:
            return None
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

# Columns to clean and convert
columns_to_clean = ['Back Squat', 'Clean and Jerk', 'Deadlift', 'Snatch']

# Apply the cleaning and conversion function to the specified columns
Women_Benchmarks[columns_to_clean] = Women_Benchmarks[columns_to_clean].applymap(clean_and_convert)

Women_Benchmarks.head()

,Athlete ID,Back Squat,Clean and Jerk,Deadlift,Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Snatch,Sprint 400m
0,1755,245.0000,165.00000,265.00000,293,NaN,4:29,2:25,11:09,NaN,25:08,120.00000,1:20
1,1775,303.0000,231.00000,345.00000,NaN,NaN,2:37,1:54,10:22,NaN,NaN,190.00000,NaN
2,1659,300.0000,235.00000,350.00000,NaN,NaN,2:15,NaN,NaN,NaN,23:51,190.00000,1:00
3,1802,250.0000,200.00000,330.00000,320,NaN,3:30,NaN,NaN,NaN,NaN,160.00000,1:23
4,1751,264.5544,213.84814,365.96692,NaN,22:54,3:39,1:52,9:11,NaN,NaN,160.93726,NaN


In [175]:
# Rename the Columns
Women_Benchmarks.rename(columns={'Back Squat': 'Back Squat (lbs)', 'Clean and Jerk':'Clean and Jerk (lbs)', 'Deadlift': 'Deadlift (lbs)', 'Snatch': 'Snatch (lbs)'}, inplace=True)
Women_Benchmarks.head()

,Athlete ID,Back Squat (lbs),Clean and Jerk (lbs),Deadlift (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Snatch (lbs),Sprint 400m
0,1755,245.0000,165.00000,265.00000,293,NaN,4:29,2:25,11:09,NaN,25:08,120.00000,1:20
1,1775,303.0000,231.00000,345.00000,NaN,NaN,2:37,1:54,10:22,NaN,NaN,190.00000,NaN
2,1659,300.0000,235.00000,350.00000,NaN,NaN,2:15,NaN,NaN,NaN,23:51,190.00000,1:00
3,1802,250.0000,200.00000,330.00000,320,NaN,3:30,NaN,NaN,NaN,NaN,160.00000,1:23
4,1751,264.5544,213.84814,365.96692,NaN,22:54,3:39,1:52,9:11,NaN,NaN,160.93726,NaN


START THE PROCESS TO JOIN HERE

In [176]:
# Remove rows with duplicate Athlete IDs from Women_Past
Women_Past_no_duplicates = Women_Past.drop_duplicates(subset='Athlete ID')

In [177]:
# Remove rows with duplicate Athlete IDs from Women_Benchmarks
Women_Benchmarks_no_duplicates = Women_Benchmarks.drop_duplicates(subset='Athlete ID')

In [178]:
# Rename Column
Women_Leaderboard.rename(columns={'CompetitorID': 'Athlete ID'}, inplace=True)

In [179]:
# Remove rows with duplicate IDs from Women_Benchmarks
Women_Leaderboard_no_duplicates = Women_Leaderboard.drop_duplicates(subset='Athlete ID')

In [180]:
# Merge Women_Past with Women_Benchmarks
merged_df = pd.merge(Women_Past_no_duplicates, Women_Benchmarks_no_duplicates, on='Athlete ID', how='inner')
merged_df.head()

,Athlete ID,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Clean and Jerk (lbs),Deadlift (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Snatch (lbs),Sprint 400m
0,1775,6060.0,1556.0,1921.0,1248.0,978.0,786.0,1203.0,7011.0,NaN,4704.0,4074.0,NaN,7217.0,NaN,303.0000,231.00000,345.00000,NaN,NaN,2:37,1:54,10:22,NaN,NaN,190.00000,NaN
1,1755,70388.0,NaN,NaN,NaN,NaN,14769.0,20353.0,14188.0,8850.0,7446.0,8116.0,12165.0,9301.0,NaN,245.0000,165.00000,265.00000,293,NaN,4:29,2:25,11:09,NaN,25:08,120.00000,1:20
2,1802,37421.0,14274.0,82750.0,17629.0,NaN,8785.0,5435.0,8117.0,7161.0,1794.0,5121.0,1126.0,1289.0,1185.0,250.0000,200.00000,330.00000,320,NaN,3:30,NaN,NaN,NaN,NaN,160.00000,1:23
3,1751,35925.0,17813.0,NaN,NaN,NaN,32938.0,53678.0,8417.0,12282.0,NaN,NaN,84.0,712.0,418.0,264.5544,213.84814,365.96692,NaN,22:54,3:39,1:52,9:11,NaN,NaN,160.93726,NaN
4,1659,3944.0,NaN,1781.0,NaN,NaN,NaN,1628.0,864.0,510.0,582.0,5986.0,1042.0,2213.0,NaN,300.0000,235.00000,350.00000,NaN,NaN,2:15,NaN,NaN,NaN,23:51,190.00000,1:00


In [181]:
# Convert 'Athlete ID' to string (object) in Women_Leaderboard
Women_Leaderboard_no_duplicates['Athlete ID'] = Women_Leaderboard_no_duplicates['Athlete ID'].astype(str)

# Merge merged_df with Men_Leaderboard
PreClean_Crossfit = pd.merge(merged_df, Women_Leaderboard_no_duplicates, on='Athlete ID', how='left')

PreClean_Crossfit.head()

,Athlete ID,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Clean and Jerk (lbs),Deadlift (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Snatch (lbs),Sprint 400m,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,1775,6060.0,1556.0,1921.0,1248.0,978.0,786.0,1203.0,7011.0,NaN,4704.0,4074.0,NaN,7217.0,NaN,303.0000,231.00000,345.00000,NaN,NaN,2:37,1:54,10:22,NaN,NaN,190.00000,NaN,Daniella Verhoest,United States,North America East,CrossFit 908,F,62.0,35.0,145.0,6060.0,8409.0,11310.0,4587.0
1,1755,70388.0,NaN,NaN,NaN,NaN,14769.0,20353.0,14188.0,8850.0,7446.0,8116.0,12165.0,9301.0,NaN,245.0000,165.00000,265.00000,293,NaN,4:29,2:25,11:09,NaN,25:08,120.00000,1:20,Robin Warnick,United States,North America East,CrossFit Terminus,F,66.0,47.0,145.0,70388.0,59325.0,109744.0,46306.0
2,1802,37421.0,14274.0,82750.0,17629.0,NaN,8785.0,5435.0,8117.0,7161.0,1794.0,5121.0,1126.0,1289.0,1185.0,250.0000,200.00000,330.00000,320,NaN,3:30,NaN,NaN,NaN,NaN,160.00000,1:23,Melissa Kiel,United States,North America East,East Ormond Beach CrossFit,F,69.0,42.0,165.0,37421.0,62869.0,36822.0,23411.0
3,1751,35925.0,17813.0,NaN,NaN,NaN,32938.0,53678.0,8417.0,12282.0,NaN,NaN,84.0,712.0,418.0,264.5544,213.84814,365.96692,NaN,22:54,3:39,1:52,9:11,NaN,NaN,160.93726,NaN,Philippa Robinson,Australia,Oceania,Ocean Fitness CrossFit,F,68.0,41.0,216.0,35925.0,60287.0,28032.0,30128.0
4,1659,3944.0,NaN,1781.0,NaN,NaN,NaN,1628.0,864.0,510.0,582.0,5986.0,1042.0,2213.0,NaN,300.0000,235.00000,350.00000,NaN,NaN,2:15,NaN,NaN,NaN,23:51,190.00000,1:00,Camas Barkemeyer,United States,North America West,CrossFit Kodiak Island,F,68.0,42.0,150.0,3944.0,11689.0,2850.0,1888.0


In [182]:
# #Output Crossfit to a CSV file
PreClean_Crossfit.to_csv('Pre-Clean_Women_Crossfit.csv', index=False)

In [183]:
Women_Clean_Me = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Pre-Clean_Women_Crossfit.csv')
Women_Clean_Me.head()

/var/folders/xm/kc32pwjj5fj5xqf43yb4dz340000gn/T/ipykernel_2907/1057959074.py:1: DtypeWarning: Columns (0,23) have mixed types. Specify dtype option on import or set low_memory=False.
  Women_Clean_Me = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Pre-Clean_Women_Crossfit.csv')


,Athlete ID,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Clean and Jerk (lbs),Deadlift (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Snatch (lbs),Sprint 400m,CompetitorName,CountryName,RegionName,AffiliateName,Gender,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank
0,1775.0,6060.0,1556.0,1921.0,1248.0,978.0,786.0,1203.0,7011.0,NaN,4704.0,4074.0,NaN,7217.0,NaN,303.0000,231.00000,345.00000,NaN,NaN,2:37,1:54,10:22,NaN,NaN,190.00000,NaN,Daniella Verhoest,United States,North America East,CrossFit 908,F,62.0,35.0,145.0,6060.0,8409.0,11310.0,4587.0
1,1755.0,70388.0,NaN,NaN,NaN,NaN,14769.0,20353.0,14188.0,8850.0,7446.0,8116.0,12165.0,9301.0,NaN,245.0000,165.00000,265.00000,293.0,NaN,4:29,2:25,11:09,NaN,25:08,120.00000,1:20,Robin Warnick,United States,North America East,CrossFit Terminus,F,66.0,47.0,145.0,70388.0,59325.0,109744.0,46306.0
2,1802.0,37421.0,14274.0,82750.0,17629.0,NaN,8785.0,5435.0,8117.0,7161.0,1794.0,5121.0,1126.0,1289.0,1185.0,250.0000,200.00000,330.00000,320.0,NaN,3:30,NaN,NaN,NaN,NaN,160.00000,1:23,Melissa Kiel,United States,North America East,East Ormond Beach CrossFit,F,69.0,42.0,165.0,37421.0,62869.0,36822.0,23411.0
3,1751.0,35925.0,17813.0,NaN,NaN,NaN,32938.0,53678.0,8417.0,12282.0,NaN,NaN,84.0,712.0,418.0,264.5544,213.84814,365.96692,NaN,22:54,3:39,1:52,9:11,NaN,NaN,160.93726,NaN,Philippa Robinson,Australia,Oceania,Ocean Fitness CrossFit,F,68.0,41.0,216.0,35925.0,60287.0,28032.0,30128.0
4,1659.0,3944.0,NaN,1781.0,NaN,NaN,NaN,1628.0,864.0,510.0,582.0,5986.0,1042.0,2213.0,NaN,300.0000,235.00000,350.00000,NaN,NaN,2:15,NaN,NaN,NaN,23:51,190.00000,1:00,Camas Barkemeyer,United States,North America West,CrossFit Kodiak Island,F,68.0,42.0,150.0,3944.0,11689.0,2850.0,1888.0


In [184]:
# Define the desired order of column names
desired_column_order = [
    'Athlete ID',
    'Gender',
    'CompetitorName',
    'CountryName',
    'RegionName',
    'AffiliateName',
    'Height (in)',
    'Age',
    'Weight (lbs)',
    'Overall Rank',
    '24.1 Rank',
    '24.2 Rank',
    '24.3 Rank',
    '2024 Rank',
    '2023 Rank',
    '2022 Rank',
    '2021 Rank',
    '2020 Rank',
    '2019 Rank',
    '2018 Rank',
    '2017 Rank',
    '2016 Rank',
    '2015 Rank',
    '2014 Rank',
    '2013 Rank',
    '2012 Rank',
    '2011 Rank',
    'Back Squat (lbs)',
    'Deadlift (lbs)',
    'Clean and Jerk (lbs)',
    'Snatch (lbs)',
    'Fight Gone Bad',
    'Filthy 50',
    'Fran',
    'Grace',
    'Helen',
    'L1 Benchmark',
    'Run 5k',
    'Sprint 400m'
 
]
Women_Clean_Me=Women_Clean_Me[desired_column_order]
Women_Clean_Me.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124928 entries, 0 to 124927
Data columns (total 39 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Athlete ID            124927 non-null  object 
 1   Gender                124926 non-null  object 
 2   CompetitorName        124924 non-null  object 
 3   CountryName           124678 non-null  object 
 4   RegionName            124926 non-null  object 
 5   AffiliateName         113581 non-null  object 
 6   Height (in)           47518 non-null   float64
 7   Age                   124926 non-null  float64
 8   Weight (lbs)          42678 non-null   float64
 9   Overall Rank          124926 non-null  float64
 10  24.1 Rank             124926 non-null  float64
 11  24.2 Rank             124926 non-null  float64
 12  24.3 Rank             124926 non-null  float64
 13  2024 Rank             124928 non-null  float64
 14  2023 Rank             66615 non-null   float64
 15  

In [185]:
# Rename the Column Names
Women_Clean_Me.rename(columns={'CompetitorName':'Competitor Name', 'CountryName':'County Name', 'RegionName':'Region Name', 'AffiliateName': 'Affiliate Name'}, inplace=True)
Women_Clean_Me.head()

,Athlete ID,Gender,Competitor Name,County Name,Region Name,Affiliate Name,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Deadlift (lbs),Clean and Jerk (lbs),Snatch (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Sprint 400m
0,1775.0,F,Daniella Verhoest,United States,North America East,CrossFit 908,62.0,35.0,145.0,6060.0,8409.0,11310.0,4587.0,6060.0,1556.0,1921.0,1248.0,978.0,786.0,1203.0,7011.0,NaN,4704.0,4074.0,NaN,7217.0,NaN,303.0000,345.00000,231.00000,190.00000,NaN,NaN,2:37,1:54,10:22,NaN,NaN,NaN
1,1755.0,F,Robin Warnick,United States,North America East,CrossFit Terminus,66.0,47.0,145.0,70388.0,59325.0,109744.0,46306.0,70388.0,NaN,NaN,NaN,NaN,14769.0,20353.0,14188.0,8850.0,7446.0,8116.0,12165.0,9301.0,NaN,245.0000,265.00000,165.00000,120.00000,293.0,NaN,4:29,2:25,11:09,NaN,25:08,1:20
2,1802.0,F,Melissa Kiel,United States,North America East,East Ormond Beach CrossFit,69.0,42.0,165.0,37421.0,62869.0,36822.0,23411.0,37421.0,14274.0,82750.0,17629.0,NaN,8785.0,5435.0,8117.0,7161.0,1794.0,5121.0,1126.0,1289.0,1185.0,250.0000,330.00000,200.00000,160.00000,320.0,NaN,3:30,NaN,NaN,NaN,NaN,1:23
3,1751.0,F,Philippa Robinson,Australia,Oceania,Ocean Fitness CrossFit,68.0,41.0,216.0,35925.0,60287.0,28032.0,30128.0,35925.0,17813.0,NaN,NaN,NaN,32938.0,53678.0,8417.0,12282.0,NaN,NaN,84.0,712.0,418.0,264.5544,365.96692,213.84814,160.93726,NaN,22:54,3:39,1:52,9:11,NaN,NaN,NaN
4,1659.0,F,Camas Barkemeyer,United States,North America West,CrossFit Kodiak Island,68.0,42.0,150.0,3944.0,11689.0,2850.0,1888.0,3944.0,NaN,1781.0,NaN,NaN,NaN,1628.0,864.0,510.0,582.0,5986.0,1042.0,2213.0,NaN,300.0000,350.00000,235.00000,190.00000,NaN,NaN,2:15,NaN,NaN,NaN,23:51,1:00


In [186]:
# Convert 'Fight Gone Bad' column to float
Women_Clean_Me['Fight Gone Bad'] = Women_Clean_Me['Fight Gone Bad'].astype(float)

In [187]:
Women_Clean_Me.head()

,Athlete ID,Gender,Competitor Name,County Name,Region Name,Affiliate Name,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Deadlift (lbs),Clean and Jerk (lbs),Snatch (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Sprint 400m
0,1775.0,F,Daniella Verhoest,United States,North America East,CrossFit 908,62.0,35.0,145.0,6060.0,8409.0,11310.0,4587.0,6060.0,1556.0,1921.0,1248.0,978.0,786.0,1203.0,7011.0,NaN,4704.0,4074.0,NaN,7217.0,NaN,303.0000,345.00000,231.00000,190.00000,NaN,NaN,2:37,1:54,10:22,NaN,NaN,NaN
1,1755.0,F,Robin Warnick,United States,North America East,CrossFit Terminus,66.0,47.0,145.0,70388.0,59325.0,109744.0,46306.0,70388.0,NaN,NaN,NaN,NaN,14769.0,20353.0,14188.0,8850.0,7446.0,8116.0,12165.0,9301.0,NaN,245.0000,265.00000,165.00000,120.00000,293.0,NaN,4:29,2:25,11:09,NaN,25:08,1:20
2,1802.0,F,Melissa Kiel,United States,North America East,East Ormond Beach CrossFit,69.0,42.0,165.0,37421.0,62869.0,36822.0,23411.0,37421.0,14274.0,82750.0,17629.0,NaN,8785.0,5435.0,8117.0,7161.0,1794.0,5121.0,1126.0,1289.0,1185.0,250.0000,330.00000,200.00000,160.00000,320.0,NaN,3:30,NaN,NaN,NaN,NaN,1:23
3,1751.0,F,Philippa Robinson,Australia,Oceania,Ocean Fitness CrossFit,68.0,41.0,216.0,35925.0,60287.0,28032.0,30128.0,35925.0,17813.0,NaN,NaN,NaN,32938.0,53678.0,8417.0,12282.0,NaN,NaN,84.0,712.0,418.0,264.5544,365.96692,213.84814,160.93726,NaN,22:54,3:39,1:52,9:11,NaN,NaN,NaN
4,1659.0,F,Camas Barkemeyer,United States,North America West,CrossFit Kodiak Island,68.0,42.0,150.0,3944.0,11689.0,2850.0,1888.0,3944.0,NaN,1781.0,NaN,NaN,NaN,1628.0,864.0,510.0,582.0,5986.0,1042.0,2213.0,NaN,300.0000,350.00000,235.00000,190.00000,NaN,NaN,2:15,NaN,NaN,NaN,23:51,1:00


Clean Each Column Individually 

In [188]:
Women_Clean_Me.columns

Index(['Athlete ID', 'Gender', 'Competitor Name', 'County Name', 'Region Name',
       'Affiliate Name', 'Height (in)', 'Age', 'Weight (lbs)', 'Overall Rank',
       '24.1 Rank', '24.2 Rank', '24.3 Rank', '2024 Rank', '2023 Rank',
       '2022 Rank', '2021 Rank', '2020 Rank', '2019 Rank', '2018 Rank',
       '2017 Rank', '2016 Rank', '2015 Rank', '2014 Rank', '2013 Rank',
       '2012 Rank', '2011 Rank', 'Back Squat (lbs)', 'Deadlift (lbs)',
       'Clean and Jerk (lbs)', 'Snatch (lbs)', 'Fight Gone Bad', 'Filthy 50',
       'Fran', 'Grace', 'Helen', 'L1 Benchmark', 'Run 5k', 'Sprint 400m'],
      dtype='object')

In [189]:
Women_Clean_Me['Athlete ID'].unique()

array([1775.0, 1755.0, 1802.0, ..., 2729635, 2730346, 2730442],
      dtype=object)

In [190]:
Women_Clean_Me['Gender'].unique()

array(['F', nan], dtype=object)

In [191]:
Women_Clean_Me['Competitor Name'].unique()

array(['Daniella Verhoest', 'Robin Warnick', 'Melissa Kiel', ...,
       'Meredith McNeil', 'Olivia Tabaka', 'Katrina Hoagland'],
      dtype=object)

In [192]:
Women_Clean_Me['County Name'].unique()

array(['United States', 'Australia', 'Canada', 'Iceland', 'New Zealand',
       'Sweden', 'United Kingdom', 'Denmark', 'Netherlands', 'Peru',
       'Brazil', 'Puerto Rico', 'Cayman Islands', 'Mexico', 'Philippines',
       'South Africa', 'Italy', 'Ecuador', 'Finland', 'Czech Republic',
       'Switzerland', 'Argentina', 'Ireland', 'Norway', 'France',
       'Korea, Republic of', 'Germany', 'Spain', 'Japan', 'Bulgaria',
       'Israel', 'Venezuela', 'Guam', 'Austria', 'Bolivia', 'India',
       'Ukraine', 'Poland', 'Greece', 'Croatia', 'Belgium', 'Namibia',
       'Viet Nam', 'Colombia', nan, 'China', 'Dominican Republic',
       'Costa Rica', 'Hong Kong, China', 'Malta', 'Samoa', 'Guatemala',
       'Barbados', 'Malaysia', 'Lithuania', 'Romania', 'Singapore',
       'Chile', 'Honduras', 'Slovenia', 'Brunei Darussalam', 'Portugal',
       'Thailand', 'United Arab Emirates', 'Virgin Islands, US',
       'Luxembourg', 'Latvia', 'Madagascar', 'Suriname', 'Iran',
       'Morocco', 'Bahrai

In [193]:
Women_Clean_Me.rename(columns={'County Name':'Country Name'}, inplace=True)

In [194]:
Women_Clean_Me['Region Name'].unique()

array(['North America East', 'Oceania', 'North America West', 'Europe',
       'South America', 'Asia', 'Africa', nan], dtype=object)

In [195]:
Women_Clean_Me['Affiliate Name'].unique()

array(['CrossFit 908', 'CrossFit Terminus', 'East Ormond Beach CrossFit',
       ..., 'CrossFit Bidart', 'CrossFit 2712 Industrieweg',
       'Canal Fulton CrossFit'], dtype=object)

In [196]:
# Count the occurrences of each unique value in the 'col1' column
value_counts = Women_Clean_Me['Height (in)'].value_counts()

# Sort the value counts in descending order
sorted_value_counts = value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(sorted_value_counts)

Height (in)
0.0           8
1.0          46
2.0          16
4.0           4
5.0         139
6.0          50
12.0          2
14.0          1
16.0          2
17.0          1
18.0          1
21.0          1
23.0          1
24.0         13
25.0          7
26.0         11
27.0          2
28.0          2
29.0          1
40.0          1
46.0          1
47.0          1
48.0          6
52.0          2
53.0          2
54.0          7
55.0          3
56.0          8
57.0         52
58.0        131
59.0        596
60.0       1686
61.0       2630
62.0       4465
63.0       5841
64.0       6444
65.0       6671
66.0       5570
67.0       5319
68.0       3147
69.0       2331
70.0       1250
71.0        503
72.0        289
73.0         56
74.0         29
75.0          6
76.0          9
77.0          3
79.0          3
80.0          1
81.0          2
82.0          1
84.0          2
96.0          1
110.0         1
114.0         1
118.0         1
128.0         1
150.0         1
153.0         1
154.0       

In [197]:
# Replace values less than 48 or greater than 77 with NaN
Women_Clean_Me.loc[(Women_Clean_Me['Height (in)'] < 48) | (Women_Clean_Me['Height (in)'] > 77), 'Height (in)'] = pd.NaT

In [198]:
Women_Clean_Me['Age'].unique()

array([35., 47., 42., 41., 49., 43., 31., 38., 45., 44., 33., 32., 52.,
       48., 37., 40., 39., 54., 34., 46., 50., 51., 36., 27., 53., 30.,
       19., 28., 29., 26., 25., nan, 22., 24., 23., 17., 21., 20., 16.,
       18.])

In [199]:
# Replace values less than 50 or greater than 450 with NaN
Women_Clean_Me.loc[(Women_Clean_Me['Weight (lbs)'] < 50) | (Women_Clean_Me['Weight (lbs)'] > 450), 'Weight (lbs)'] = pd.NaT

In [200]:
# Count the occurrences of each unique value in the 'col1' column
weight_value_counts = Women_Clean_Me['Weight (lbs)'].value_counts()

# Sort the value counts in descending order
sorted_value_counts = weight_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(sorted_value_counts)

Weight (lbs)
50.0        1
51.0        2
52.0        5
53.0        5
54.0       10
55.0       12
56.0        5
57.0       26
58.0        9
59.0        5
60.0       32
61.0        8
62.0       14
63.0       17
64.0       13
65.0       13
66.0        7
67.0       11
68.0       16
69.0        8
70.0       14
71.0        7
72.0        4
73.0        6
74.0        1
75.0        4
77.0        3
78.0        2
79.0        1
80.0        5
83.0        1
85.0        2
86.0        1
87.0        2
88.0        6
90.0        5
93.0        2
94.0        1
95.0       24
96.0        1
97.0       18
98.0       11
99.0       32
100.0      46
101.0      32
102.0      16
103.0      17
104.0      82
105.0     119
106.0     107
107.0      47
108.0     131
109.0      18
110.0     474
111.0      19
112.0     274
113.0      69
114.0      83
115.0     827
116.0     103
117.0     544
118.0     215
119.0     502
120.0     794
121.0     764
122.0     182
123.0     792
124.0     143
125.0    1168
126.0     809
127.0  

In [201]:
Women_Clean_Me['Overall Rank'].unique()

array([  6060.,  70388.,  37421., ...,  81345., 134650., 114884.])

In [202]:
Women_Clean_Me['24.3 Rank'].unique()

array([  4587.,  46306.,  23411., ..., 114963.,  73040.,  75335.])

In [203]:
# Replace values less than 45 or greater than 600 with NaN
Women_Clean_Me.loc[(Women_Clean_Me['Back Squat (lbs)'] < 45) | (Women_Clean_Me['Back Squat (lbs)'] > 600), 'Back Squat (lbs)'] = pd.NaT

In [204]:
# Count the occurrences of each unique value in the 'col1' column
BS_value_counts = Women_Clean_Me['Back Squat (lbs)'].value_counts()

# Sort the value counts in descending order
BS_sorted_value_counts = BS_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(BS_sorted_value_counts)

Back Squat (lbs)
45.0                    2
50.0                    3
52.0                    1
53.0                    1
55.0                    7
55.1155                 3
57.0                    1
58.0                    1
61.0                    1
61.72935999999999       2
65.0                   12
66.1386                 9
67.0                    1
69.0                    2
70.0                   10
70.54784                4
72.0                    1
72.75246                1
73.0                    1
74.95707999999999       1
75.0                   14
77.0                    2
77.1617                28
80.0                    9
81.57094                2
82.0                    1
83.77556                6
85.0                   17
85.98017999999999       1
88.0                    1
88.1848                38
90.0                   14
90.38942                3
91.0                    1
92.59404               12
93.0                    1
94.79866                6
95.0                 

In [205]:
# Count the occurrences of each unique value in the 'col1' column
DL_value_counts = Women_Clean_Me['Deadlift (lbs)'].value_counts()

# Sort the value counts in descending order
DL_sorted_value_counts = DL_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(DL_sorted_value_counts)

Deadlift (lbs)
1.00000         5
2.00000         2
2.20462        12
3.00000         3
4.40924         1
7.00000         2
8.00000         1
8.81848         1
11.02310        1
25.00000        1
37.47854        1
43.00000        2
44.09240        1
50.00000        3
55.00000        1
55.11550        3
65.00000        4
70.00000        3
74.00000        1
74.95708        1
75.00000        2
78.00000        1
80.00000        5
81.57094        1
83.77556        1
85.00000        6
88.18480        4
90.00000        8
90.38942        1
92.00000        1
92.59404        2
94.00000        1
94.79866       17
95.00000       25
99.20790       16
100.00000       8
102.00000       3
103.00000       1
103.61714       2
105.00000      23
105.82176       2
107.00000       1
110.00000      11
110.23100      24
112.00000       1
112.43562       1
114.64024       4
115.00000      20
116.84486       4
117.00000       2
118.00000       1
120.00000      15
121.25410      35
123.45872      10
124.00000    

In [206]:
# Replace values less than 44 or greater than 500 with NaN
Women_Clean_Me.loc[(Women_Clean_Me['Clean and Jerk (lbs)'] < 44) | (Women_Clean_Me['Clean and Jerk (lbs)'] > 500), 'Clean and Jerk (lbs)'] = pd.NaT

In [207]:
# Count the occurrences of each unique value in the 'col1' column
CJ_value_counts = Women_Clean_Me['Clean and Jerk (lbs)'].value_counts()

# Sort the value counts in descending order
CJ_sorted_value_counts = CJ_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(CJ_sorted_value_counts)

Clean and Jerk (lbs)
44.0924                 10
45.0                    13
46.0                     2
46.29702                 1
47.0                     3
48.50164                 3
50.0                     7
50.70625999999999        3
52.0                     2
52.91087999999999        2
53.0                     4
55.0                    28
55.1155                 21
57.32012                 1
58.0                     2
59.0                     1
59.52474                 9
60.0                    16
61.72935999999999        5
62.0                     3
63.93397999999999        6
65.0                    46
66.0                     1
66.1386                 63
67.0                     1
68.0                     5
68.34321999999999        5
70.0                    40
70.54784                25
72.0                     6
72.75246                15
73.0                     1
74.95707999999999        3
75.0                    77
76.0                     1
77.1617                106
78.0   

In [208]:
# Replace values less than 44 or greater than 450 with NaN
Women_Clean_Me.loc[(Women_Clean_Me['Snatch (lbs)'] < 44) | (Women_Clean_Me['Snatch (lbs)'] > 450), 'Snatch (lbs)'] = pd.NaT

In [209]:
# Count the occurrences of each unique value in the 'col1' column
SN_value_counts = Women_Clean_Me['Snatch (lbs)'].value_counts()

# Sort the value counts in descending order
SN_sorted_value_counts = SN_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(SN_sorted_value_counts)

Snatch (lbs)
44.0                     1
44.0924                 38
45.0                    59
46.0                     1
46.29702                 4
47.0                     1
48.0                     3
48.50164                16
49.0                     1
50.0                    33
50.70625999999999       14
51.0                     1
52.0                     5
52.91087999999999        3
53.0                     4
54.0                     1
55.0                   115
55.1155                121
56.0                     1
57.0                     6
57.32012                11
58.0                     3
59.0                     1
59.52474                41
60.0                    57
61.0                     2
61.72935999999999       32
62.0                     4
63.0                     7
63.93397999999999       29
64.0                     1
65.0                   270
66.0                     5
66.1386                255
67.0                     8
68.0                     3
68.343219999999

In [210]:
# Replace values greater than 550 with NaN
Women_Clean_Me.loc[(Women_Clean_Me['Fight Gone Bad'] > 450), 'Fight Gone Bad'] = pd.NaT

In [211]:
# Count the occurrences of each unique value in the 'col1' column
FGB_value_counts = Women_Clean_Me['Fight Gone Bad'].value_counts()

# Sort the value counts in descending order
FGB_sorted_value_counts = FGB_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(FGB_sorted_value_counts)

Fight Gone Bad
1.0       6
2.0       2
3.0       1
4.0       1
5.0       2
6.0       1
10.0      1
25.0      1
73.0      1
98.0      1
100.0     3
105.0     1
108.0     1
115.0     1
118.0     1
125.0     3
132.0     2
136.0     1
139.0     1
141.0     1
143.0     1
144.0     2
145.0     1
146.0     1
147.0     2
148.0     2
149.0     1
150.0     6
151.0     1
152.0     1
154.0     1
155.0     1
156.0     2
157.0     1
158.0     6
159.0     2
160.0     4
161.0     1
162.0     2
163.0     4
164.0     4
165.0     1
166.0     6
167.0     5
168.0     2
169.0     4
170.0     3
171.0     6
172.0     2
173.0     4
174.0     4
175.0     8
176.0     4
177.0     3
178.0     8
179.0     5
180.0     4
181.0     4
182.0     5
183.0     5
184.0     5
185.0    11
186.0     9
187.0     8
188.0     8
189.0    12
190.0     9
191.0    11
192.0     6
193.0     5
194.0     7
195.0     4
196.0     5
197.0     8
198.0    17
199.0     9
200.0    18
201.0    10
202.0     7
203.0    20
204.0    11
205.0     9
2

In [212]:
#Output Crossfit to a CSV file
Women_Clean_Me.to_csv('Pre_DT_Women_Crossfit.csv', index=False)

In [213]:
Pre_DT = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Pre_DT_Women_Crossfit.csv')
Pre_DT.head()

/var/folders/xm/kc32pwjj5fj5xqf43yb4dz340000gn/T/ipykernel_2907/1272570959.py:1: DtypeWarning: Columns (0,36) have mixed types. Specify dtype option on import or set low_memory=False.
  Pre_DT = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Pre_DT_Women_Crossfit.csv')


,Athlete ID,Gender,Competitor Name,Country Name,Region Name,Affiliate Name,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Deadlift (lbs),Clean and Jerk (lbs),Snatch (lbs),Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Run 5k,Sprint 400m
0,1775.0,F,Daniella Verhoest,United States,North America East,CrossFit 908,62.0,35.0,145.0,6060.0,8409.0,11310.0,4587.0,6060.0,1556.0,1921.0,1248.0,978.0,786.0,1203.0,7011.0,NaN,4704.0,4074.0,NaN,7217.0,NaN,303.0000,345.00000,231.00000,190.00000,NaN,NaN,2:37,1:54,10:22,NaN,NaN,NaN
1,1755.0,F,Robin Warnick,United States,North America East,CrossFit Terminus,66.0,47.0,145.0,70388.0,59325.0,109744.0,46306.0,70388.0,NaN,NaN,NaN,NaN,14769.0,20353.0,14188.0,8850.0,7446.0,8116.0,12165.0,9301.0,NaN,245.0000,265.00000,165.00000,120.00000,293.0,NaN,4:29,2:25,11:09,NaN,25:08,1:20
2,1802.0,F,Melissa Kiel,United States,North America East,East Ormond Beach CrossFit,69.0,42.0,165.0,37421.0,62869.0,36822.0,23411.0,37421.0,14274.0,82750.0,17629.0,NaN,8785.0,5435.0,8117.0,7161.0,1794.0,5121.0,1126.0,1289.0,1185.0,250.0000,330.00000,200.00000,160.00000,320.0,NaN,3:30,NaN,NaN,NaN,NaN,1:23
3,1751.0,F,Philippa Robinson,Australia,Oceania,Ocean Fitness CrossFit,68.0,41.0,216.0,35925.0,60287.0,28032.0,30128.0,35925.0,17813.0,NaN,NaN,NaN,32938.0,53678.0,8417.0,12282.0,NaN,NaN,84.0,712.0,418.0,264.5544,365.96692,213.84814,160.93726,NaN,22:54,3:39,1:52,9:11,NaN,NaN,NaN
4,1659.0,F,Camas Barkemeyer,United States,North America West,CrossFit Kodiak Island,68.0,42.0,150.0,3944.0,11689.0,2850.0,1888.0,3944.0,NaN,1781.0,NaN,NaN,NaN,1628.0,864.0,510.0,582.0,5986.0,1042.0,2213.0,NaN,300.0000,350.00000,235.00000,190.00000,NaN,NaN,2:15,NaN,NaN,NaN,23:51,1:00


In [214]:
# Columns to be converted to datetime
time_columns = ['Filthy 50', 'Fran', 'Grace', 'Helen', 'L1 Benchmark', 'Run 5k', 'Sprint 400m']

# Function to convert mm:ss format to total seconds
def convert_to_seconds(value):
    if pd.isnull(value):
        return pd.NaT
    else:
        minutes, seconds = map(int, value.split(':'))
        return minutes * 60 + seconds

# Apply conversion to each column and add new columns with total seconds
for col in time_columns:
    Pre_DT[col + '_total_seconds'] = Pre_DT[col].apply(convert_to_seconds)

In [215]:
desired_columns_order = [
    "Athlete ID",
    "Gender",
    "Competitor Name",
    "Country Name",
    "Region Name",
    "Affiliate Name",
    "Height (in)",
    "Age",
    "Weight (lbs)",
    "Overall Rank",
    "24.1 Rank",
    "24.2 Rank",
    "24.3 Rank",
    "2024 Rank",
    "2023 Rank",
    "2022 Rank",
    "2021 Rank",
    "2020 Rank",
    "2019 Rank",
    "2018 Rank",
    "2017 Rank",
    "2016 Rank",
    "2015 Rank",
    "2014 Rank",
    "2013 Rank",
    "2012 Rank",
    "2011 Rank",
    "Back Squat (lbs)",
    "Deadlift (lbs)",
    "Clean and Jerk (lbs)",
    "Snatch (lbs)",
    "Fight Gone Bad",
    "Filthy 50",
    "Filthy 50_total_seconds",
    "Fran",
    "Fran_total_seconds",
    "Grace",
    "Grace_total_seconds",
    "Helen",
    "Helen_total_seconds",
    "L1 Benchmark",
    "L1 Benchmark_total_seconds",
    "Run 5k",
    "Run 5k_total_seconds",
    "Sprint 400m",
    "Sprint 400m_total_seconds"

]

# Reorder the columns
Pre_DT = Pre_DT[desired_columns_order]

In [216]:
Pre_DT.head()

,Athlete ID,Gender,Competitor Name,Country Name,Region Name,Affiliate Name,Height (in),Age,Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Deadlift (lbs),Clean and Jerk (lbs),Snatch (lbs),Fight Gone Bad,Filthy 50,Filthy 50_total_seconds,Fran,Fran_total_seconds,Grace,Grace_total_seconds,Helen,Helen_total_seconds,L1 Benchmark,L1 Benchmark_total_seconds,Run 5k,Run 5k_total_seconds,Sprint 400m,Sprint 400m_total_seconds
0,1775.0,F,Daniella Verhoest,United States,North America East,CrossFit 908,62.0,35.0,145.0,6060.0,8409.0,11310.0,4587.0,6060.0,1556.0,1921.0,1248.0,978.0,786.0,1203.0,7011.0,NaN,4704.0,4074.0,NaN,7217.0,NaN,303.0000,345.00000,231.00000,190.00000,NaN,NaN,NaT,2:37,157,1:54,114,10:22,622,NaN,NaT,NaN,NaT,NaN,NaT
1,1755.0,F,Robin Warnick,United States,North America East,CrossFit Terminus,66.0,47.0,145.0,70388.0,59325.0,109744.0,46306.0,70388.0,NaN,NaN,NaN,NaN,14769.0,20353.0,14188.0,8850.0,7446.0,8116.0,12165.0,9301.0,NaN,245.0000,265.00000,165.00000,120.00000,293.0,NaN,NaT,4:29,269,2:25,145,11:09,669,NaN,NaT,25:08,1508,1:20,80
2,1802.0,F,Melissa Kiel,United States,North America East,East Ormond Beach CrossFit,69.0,42.0,165.0,37421.0,62869.0,36822.0,23411.0,37421.0,14274.0,82750.0,17629.0,NaN,8785.0,5435.0,8117.0,7161.0,1794.0,5121.0,1126.0,1289.0,1185.0,250.0000,330.00000,200.00000,160.00000,320.0,NaN,NaT,3:30,210,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,1:23,83
3,1751.0,F,Philippa Robinson,Australia,Oceania,Ocean Fitness CrossFit,68.0,41.0,216.0,35925.0,60287.0,28032.0,30128.0,35925.0,17813.0,NaN,NaN,NaN,32938.0,53678.0,8417.0,12282.0,NaN,NaN,84.0,712.0,418.0,264.5544,365.96692,213.84814,160.93726,NaN,22:54,1374,3:39,219,1:52,112,9:11,551,NaN,NaT,NaN,NaT,NaN,NaT
4,1659.0,F,Camas Barkemeyer,United States,North America West,CrossFit Kodiak Island,68.0,42.0,150.0,3944.0,11689.0,2850.0,1888.0,3944.0,NaN,1781.0,NaN,NaN,NaN,1628.0,864.0,510.0,582.0,5986.0,1042.0,2213.0,NaN,300.0000,350.00000,235.00000,190.00000,NaN,NaN,NaT,2:15,135,NaN,NaT,NaN,NaT,NaN,NaT,23:51,1431,1:00,60


In [217]:
# Rename Column
Pre_DT.rename(columns={'Filthy 50': 'Filthy 50 (time)', 'Filthy 50_total_seconds':'Filthy 50 (sec)', 'Fran':'Fran (time)', 'Fran_total_seconds':'Fran (sec)', 'Grace':'Grace (time)', 'Grace_total_seconds': 'Grace (sec)', 'Helen': 'Helen (time)', 'Helen_total_seconds':'Helen (sec)', 'Helen_total_seconds': 'Helen (sec)', 'L1 Benchmark': 'L1 Benchmark (time)', 'L1 Benchmark_total_seconds':'L1 Benchmark (sec)', 'Run 5k': 'Run 5k (time)', 'Run 5k_total_seconds':'Run 5k (sec)', 'Sprint 400m': 'Sprint 400m (time)', 'Sprint 400m_total_seconds':'Sprint 400m (sec)', 'Age':'Age (Years)', 'Fight Gone Bad':'Fight Gone Bad (# reps)'}, inplace=True)

In [218]:
Pre_DT.head()

,Athlete ID,Gender,Competitor Name,Country Name,Region Name,Affiliate Name,Height (in),Age (Years),Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Deadlift (lbs),Clean and Jerk (lbs),Snatch (lbs),Fight Gone Bad (# reps),Filthy 50 (time),Filthy 50 (sec),Fran (time),Fran (sec),Grace (time),Grace (sec),Helen (time),Helen (sec),L1 Benchmark (time),L1 Benchmark (sec),Run 5k (time),Run 5k (sec),Sprint 400m (time),Sprint 400m (sec)
0,1775.0,F,Daniella Verhoest,United States,North America East,CrossFit 908,62.0,35.0,145.0,6060.0,8409.0,11310.0,4587.0,6060.0,1556.0,1921.0,1248.0,978.0,786.0,1203.0,7011.0,NaN,4704.0,4074.0,NaN,7217.0,NaN,303.0000,345.00000,231.00000,190.00000,NaN,NaN,NaT,2:37,157,1:54,114,10:22,622,NaN,NaT,NaN,NaT,NaN,NaT
1,1755.0,F,Robin Warnick,United States,North America East,CrossFit Terminus,66.0,47.0,145.0,70388.0,59325.0,109744.0,46306.0,70388.0,NaN,NaN,NaN,NaN,14769.0,20353.0,14188.0,8850.0,7446.0,8116.0,12165.0,9301.0,NaN,245.0000,265.00000,165.00000,120.00000,293.0,NaN,NaT,4:29,269,2:25,145,11:09,669,NaN,NaT,25:08,1508,1:20,80
2,1802.0,F,Melissa Kiel,United States,North America East,East Ormond Beach CrossFit,69.0,42.0,165.0,37421.0,62869.0,36822.0,23411.0,37421.0,14274.0,82750.0,17629.0,NaN,8785.0,5435.0,8117.0,7161.0,1794.0,5121.0,1126.0,1289.0,1185.0,250.0000,330.00000,200.00000,160.00000,320.0,NaN,NaT,3:30,210,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,1:23,83
3,1751.0,F,Philippa Robinson,Australia,Oceania,Ocean Fitness CrossFit,68.0,41.0,216.0,35925.0,60287.0,28032.0,30128.0,35925.0,17813.0,NaN,NaN,NaN,32938.0,53678.0,8417.0,12282.0,NaN,NaN,84.0,712.0,418.0,264.5544,365.96692,213.84814,160.93726,NaN,22:54,1374,3:39,219,1:52,112,9:11,551,NaN,NaT,NaN,NaT,NaN,NaT
4,1659.0,F,Camas Barkemeyer,United States,North America West,CrossFit Kodiak Island,68.0,42.0,150.0,3944.0,11689.0,2850.0,1888.0,3944.0,NaN,1781.0,NaN,NaN,NaN,1628.0,864.0,510.0,582.0,5986.0,1042.0,2213.0,NaN,300.0000,350.00000,235.00000,190.00000,NaN,NaN,NaT,2:15,135,NaN,NaT,NaN,NaT,NaN,NaT,23:51,1431,1:00,60


In [219]:
# Replace values less than 300 or greater than 3600 with NaN
Pre_DT.loc[(Pre_DT['Filthy 50 (sec)'] < 300) | (Pre_DT['Filthy 50 (sec)'] > 3600), 'Filthy 50 (sec)'] = pd.NaT

# Set 'Filthy 50 (time)' to NaN for rows where 'Filthy 50 (sec)' is NaN
Pre_DT.loc[Pre_DT['Filthy 50 (sec)'].isna(), 'Filthy 50 (time)'] = pd.NaT

In [220]:
# Count the occurrences of each unique value in the 'col1' column
F50_value_counts = Pre_DT['Filthy 50 (time)'].value_counts()

# Sort the value counts in descending order
F50_sorted_value_counts = F50_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(F50_sorted_value_counts)

Filthy 50 (time)
10:00     1
12:00     2
12:08     1
12:30     1
13:59     1
14:16     1
14:41     1
14:52     1
15:04     1
15:07     1
15:08     1
15:14     1
15:16     1
15:34     1
15:35     1
15:47     1
15:57     1
16:00     1
16:02     1
16:05     1
16:11     1
16:12     1
16:16     1
16:27     1
16:33     1
16:40     1
16:45     1
16:47     1
16:49     1
16:56     1
16:57     1
17:00     1
17:04     1
17:09     1
17:14     1
17:17     1
17:18     1
17:21     2
17:23     1
17:28     1
17:30     1
17:33     1
17:34     1
17:36     2
17:37     1
17:38     1
17:44     2
17:45     1
17:46     3
17:47     1
17:50     3
17:53     1
17:55     1
18:00     3
18:03     2
18:04     1
18:06     1
18:08     4
18:12     1
18:15     1
18:16     2
18:17     3
18:18     1
18:19     3
18:20     2
18:25     1
18:26     1
18:31     2
18:33     2
18:35     1
18:36     4
18:37     1
18:39     1
18:41     1
18:45     2
18:46     2
18:47     2
18:48     1
18:50     1
18:51     2
18:52     2
18:53     1

In [221]:
# Replace values less than 105 or greater than 900 with NaN
Pre_DT.loc[(Pre_DT['Fran (sec)'] < 105) | (Pre_DT['Fran (sec)'] > 900), 'Fran (sec)'] = pd.NaT

# Set 'Fran (time)' to NaN for rows where 'Fran (sec)' is NaN
Pre_DT.loc[Pre_DT['Fran (sec)'].isna(), 'Fran (time)'] = pd.NaT

In [222]:
# Count the occurrences of each unique value in the 'col1' column
Fran_value_counts = Pre_DT['Fran (time)'].value_counts()

# Sort the value counts in descending order
Fran_sorted_value_counts = Fran_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(Fran_sorted_value_counts)

Fran (time)
10:00    21
10:01     3
10:02     1
10:03     3
10:04     1
10:05     3
10:07     2
10:08     3
10:09     1
10:10     2
10:11     1
10:12     2
10:13     5
10:14     4
10:15     3
10:16     1
10:17     1
10:18     3
10:19     1
10:20     1
10:22     1
10:23     1
10:24     1
10:25     4
10:26     1
10:28     1
10:29     3
10:30     2
10:32     3
10:33     4
10:34     3
10:35     3
10:37     1
10:38     3
10:39     2
10:40     2
10:44     1
10:45     2
10:46     1
10:47     2
10:49     1
10:50     1
10:51     1
10:54     1
10:57     2
10:58     2
11:00     6
11:01     1
11:02     1
11:04     2
11:06     1
11:07     2
11:08     2
11:11     4
11:13     3
11:15     2
11:16     1
11:17     2
11:19     3
11:20     2
11:22     1
11:23     2
11:24     2
11:28     1
11:35     2
11:36     1
11:37     2
11:38     1
11:39     2
11:40     2
11:41     1
11:43     1
11:45     3
11:46     1
11:47     2
11:48     1
11:49     1
11:50     1
11:51     1
11:52     1
11:53     1
11:54     1
11:5

In [223]:
# Replace values less than 59 or greater than 1800 with NaN
Pre_DT.loc[(Pre_DT['Grace (sec)'] < 59) | (Pre_DT['Grace (sec)'] > 1800), 'Grace (sec)'] = pd.NaT

# Set 'Grace (time)' to NaN for rows where 'Grace (sec)' is NaN
Pre_DT.loc[Pre_DT['Grace (sec)'].isna(), 'Grace (time)'] = pd.NaT

In [224]:
# Count the occurrences of each unique value in the 'col1' column
Grace_value_counts = Pre_DT['Grace (time)'].value_counts()

# Sort the value counts in descending order
Grace_sorted_value_counts = Grace_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(Grace_sorted_value_counts)

Grace (time)
0:59      1
10:00     2
10:04     2
10:12     1
10:28     1
10:34     1
10:36     1
10:38     1
10:45     2
10:52     1
10:58     1
11:00     1
11:02     1
11:08     1
11:11     1
11:28     1
11:37     1
11:40     1
11:57     1
12:00     2
12:04     1
12:28     1
12:38     1
12:45     1
12:55     2
13:00     1
13:15     1
13:53     2
14:08     1
14:52     1
15:07     1
15:22     1
15:41     1
18:59     1
19:23     1
1:00      1
1:02      1
1:03      1
1:04      1
1:07      1
1:09      2
1:10      2
1:11      1
1:12      2
1:13      2
1:14      1
1:15      2
1:16      4
1:17      7
1:18      7
1:19      3
1:20      4
1:21      9
1:22      4
1:23      1
1:24      7
1:25      3
1:26      5
1:27      6
1:28      8
1:29     12
1:30      5
1:31      7
1:32      9
1:33     12
1:34      9
1:35      9
1:36     11
1:37      7
1:38     11
1:39     14
1:40     13
1:41      9
1:42     17
1:43     23
1:44     12
1:45     21
1:46     15
1:47     23
1:48     32
1:49     18
1:50     29
1:5

In [225]:
# Replace values less than 300 or greater than 1800 with NaN
Pre_DT.loc[(Pre_DT['Helen (sec)'] < 300) | (Pre_DT['Helen (sec)'] > 1800), 'Helen (sec)'] = pd.NaT

# Set 'Helen (time)' to NaN for rows where 'Helen (sec)' is NaN
Pre_DT.loc[Pre_DT['Helen (sec)'].isna(), 'Helen (time)'] = pd.NaT

In [226]:
# Count the occurrences of each unique value in the 'col1' column
Helen_value_counts = Pre_DT['Helen (time)'].value_counts()

# Sort the value counts in descending order
Helen_sorted_value_counts = Helen_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(Helen_sorted_value_counts)

Helen (time)
10:00    24
10:01    11
10:02    20
10:03    17
10:04    16
10:05    16
10:06    15
10:07    19
10:08    21
10:09     9
10:10    22
10:11    13
10:12    11
10:13    20
10:14    11
10:15    27
10:16    20
10:17    20
10:18    16
10:19    18
10:20    16
10:21    22
10:22    19
10:23    16
10:24    20
10:25    16
10:26    20
10:27    22
10:28    12
10:29    15
10:30    31
10:31    16
10:32    23
10:33    16
10:34    19
10:35    19
10:36    17
10:37    21
10:38    24
10:39    16
10:40    18
10:41    15
10:42    13
10:43    20
10:44    17
10:45    18
10:46    19
10:47     9
10:48    27
10:49     9
10:50    25
10:51    20
10:52    21
10:53    20
10:54    23
10:55    24
10:56    30
10:57    15
10:58    17
10:59    19
11:00    35
11:01     8
11:02    15
11:03    18
11:04    14
11:05    19
11:06    16
11:07    27
11:08    21
11:09    18
11:10    16
11:11    14
11:12    12
11:13    13
11:14    16
11:15    20
11:16    21
11:17    19
11:18    18
11:19    16
11:20    21
11:21    16
11:

In [227]:
# Replace values less than 60 or greater than 1800 with NaN
Pre_DT.loc[(Pre_DT['L1 Benchmark (sec)'] < 60) | (Pre_DT['L1 Benchmark (sec)'] > 1800), 'L1 Benchmark (sec)'] = pd.NaT

# Set 'L1 Benchmark (time)' to NaN for rows where 'Helen (sec)' is NaN
Pre_DT.loc[Pre_DT['L1 Benchmark (sec)'].isna(), 'L1 Benchmark (time)'] = pd.NaT

In [228]:
# Count the occurrences of each unique value in the 'col1' column
L1_value_counts = Pre_DT['L1 Benchmark (time)'].value_counts()

# Sort the value counts in descending order
L1_sorted_value_counts = L1_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(L1_sorted_value_counts)

L1 Benchmark (time)
10:40    1
10:48    1
10:52    1
11:00    1
11:17    1
11:30    1
14:39    1
15:00    1
15:48    1
1:00     2
20:00    1
30:00    1
3:38     1
3:39     1
3:46     1
3:47     1
3:48     1
3:49     1
3:52     1
3:56     1
3:59     1
4:04     1
4:10     2
4:11     2
4:12     3
4:13     1
4:15     1
4:20     1
4:21     1
4:23     1
4:24     1
4:26     1
4:27     2
4:28     1
4:33     1
4:34     2
4:37     1
4:39     1
4:40     1
4:43     4
4:44     2
4:45     1
4:48     1
4:50     1
4:51     1
4:52     2
4:53     3
4:54     1
4:55     1
5:00     2
5:02     1
5:03     1
5:04     1
5:05     2
5:06     1
5:07     1
5:10     2
5:12     1
5:14     1
5:15     1
5:17     1
5:18     1
5:20     1
5:23     1
5:25     2
5:26     1
5:28     1
5:30     2
5:31     2
5:34     1
5:35     1
5:36     1
5:37     1
5:46     1
5:50     2
5:53     3
5:55     2
5:56     3
5:57     1
5:58     2
5:59     1
6:00     2
6:01     1
6:03     3
6:08     1
6:09     1
6:10     1
6:12     2
6:14     1
6

In [229]:
# Replace values less than 600 or greater than 1800 with NaN
Pre_DT.loc[(Pre_DT['Run 5k (sec)'] < 600) | (Pre_DT['Run 5k (sec)'] > 3600), 'Run 5k (sec)'] = pd.NaT

# Set 'L1 Benchmark (time)' to NaN for rows where 'Helen (sec)' is NaN
Pre_DT.loc[Pre_DT['Run 5k (sec)'].isna(), 'Run 5k (time)'] = pd.NaT

In [230]:
# Count the occurrences of each unique value in the 'col1' column
Run5k_value_counts = Pre_DT['Run 5k (time)'].value_counts()

# Sort the value counts in descending order
Run5k_sorted_value_counts = Run5k_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(Run5k_sorted_value_counts)

Run 5k (time)
10:00      1
10:30      1
11:08      1
11:45      1
12:11      1
13:54      1
14:15      1
15:00      1
15:05      1
15:32      1
15:40      1
16:23      1
16:32      1
16:59      1
17:00      2
17:17      1
17:26      1
17:30      4
17:38      1
17:41      1
17:43      1
17:45      2
17:47      1
17:49      1
17:53      1
17:56      1
17:59      1
18:00      6
18:02      2
18:03      1
18:04      1
18:07      1
18:09      1
18:10      1
18:11      1
18:13      1
18:15      2
18:16      2
18:20      2
18:22      1
18:23      1
18:24      1
18:26      1
18:30      5
18:31      1
18:32      2
18:33      2
18:34      1
18:36      1
18:38      1
18:40      1
18:45      2
18:46      1
18:47      2
18:48      2
18:49      2
18:50      3
18:51      2
18:52      1
18:53      1
18:55      1
18:56      1
18:57      1
18:58      1
19:00     10
19:02      1
19:03      2
19:05      2
19:06      2
19:07      1
19:08      4
19:09      1
19:11      3
19:12      1
19:15      1
19:17      

In [231]:
# Replace values less than 43 or greater than 600 with NaN
Pre_DT.loc[(Pre_DT['Sprint 400m (sec)'] < 43) | (Pre_DT['Sprint 400m (sec)'] > 600), 'Sprint 400m (sec)'] = pd.NaT

# Set 'L1 Benchmark (time)' to NaN for rows where 'Helen (sec)' is NaN
Pre_DT.loc[Pre_DT['Sprint 400m (sec)'].isna(), 'Sprint 400m (time)'] = pd.NaT

In [232]:
# Count the occurrences of each unique value in the 'col1' column
sprint_value_counts = Pre_DT['Sprint 400m (time)'].value_counts()

# Sort the value counts in descending order
sprint_sorted_value_counts = sprint_value_counts.sort_index()

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the sorted counts
print(sprint_sorted_value_counts)

Sprint 400m (time)
0:43       2
0:45       4
0:49       1
0:50       3
0:52       5
0:54       3
0:55       5
0:56      13
0:57      17
0:58      23
0:59      23
10:00      1
1:00      44
1:01      19
1:02      39
1:03      26
1:04      38
1:05      46
1:06      34
1:07      43
1:08      24
1:09      45
1:10      71
1:11      21
1:12      55
1:13      27
1:14      58
1:15      91
1:16      49
1:17      62
1:18      70
1:19      64
1:20     112
1:21      53
1:22      57
1:23      59
1:24      47
1:25      86
1:26      50
1:27      55
1:28      77
1:29      46
1:30     146
1:31      53
1:32      62
1:33      48
1:34      47
1:35      97
1:36      43
1:37      50
1:38      47
1:39      37
1:40     112
1:41      32
1:42      36
1:43      38
1:44      19
1:45      98
1:46      32
1:47      34
1:48      26
1:49      16
1:50      61
1:51      16
1:52      17
1:53      28
1:54      18
1:55      26
1:56      22
1:57      19
1:58      22
1:59      17
2:00      91
2:01      10
2:02      13
2:03  

In [233]:
Pre_DT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124928 entries, 0 to 124927
Data columns (total 46 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Athlete ID               124927 non-null  object 
 1   Gender                   124926 non-null  object 
 2   Competitor Name          124924 non-null  object 
 3   Country Name             124678 non-null  object 
 4   Region Name              124926 non-null  object 
 5   Affiliate Name           113581 non-null  object 
 6   Height (in)              47056 non-null   float64
 7   Age (Years)              124926 non-null  float64
 8   Weight (lbs)             42626 non-null   float64
 9   Overall Rank             124926 non-null  float64
 10  24.1 Rank                124926 non-null  float64
 11  24.2 Rank                124926 non-null  float64
 12  24.3 Rank                124926 non-null  float64
 13  2024 Rank                124928 non-null  float64
 14  2023

In [234]:
# Filter columns by data type
datetime_columns = Pre_DT.select_dtypes(include=['datetime64']).columns

# Drop the column 'Fran 50 (sec)' if it's in the datetime columns list
if 'Fran 50 (sec)' in datetime_columns:
    Pre_DT.drop(columns=['Fran 50 (sec)'], inplace=True)

In [235]:
Pre_DT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124928 entries, 0 to 124927
Data columns (total 46 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Athlete ID               124927 non-null  object 
 1   Gender                   124926 non-null  object 
 2   Competitor Name          124924 non-null  object 
 3   Country Name             124678 non-null  object 
 4   Region Name              124926 non-null  object 
 5   Affiliate Name           113581 non-null  object 
 6   Height (in)              47056 non-null   float64
 7   Age (Years)              124926 non-null  float64
 8   Weight (lbs)             42626 non-null   float64
 9   Overall Rank             124926 non-null  float64
 10  24.1 Rank                124926 non-null  float64
 11  24.2 Rank                124926 non-null  float64
 12  24.3 Rank                124926 non-null  float64
 13  2024 Rank                124928 non-null  float64
 14  2023

In [236]:
Pre_DT.head()

,Athlete ID,Gender,Competitor Name,Country Name,Region Name,Affiliate Name,Height (in),Age (Years),Weight (lbs),Overall Rank,24.1 Rank,24.2 Rank,24.3 Rank,2024 Rank,2023 Rank,2022 Rank,2021 Rank,2020 Rank,2019 Rank,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,2013 Rank,2012 Rank,2011 Rank,Back Squat (lbs),Deadlift (lbs),Clean and Jerk (lbs),Snatch (lbs),Fight Gone Bad (# reps),Filthy 50 (time),Filthy 50 (sec),Fran (time),Fran (sec),Grace (time),Grace (sec),Helen (time),Helen (sec),L1 Benchmark (time),L1 Benchmark (sec),Run 5k (time),Run 5k (sec),Sprint 400m (time),Sprint 400m (sec)
0,1775.0,F,Daniella Verhoest,United States,North America East,CrossFit 908,62.0,35.0,145.0,6060.0,8409.0,11310.0,4587.0,6060.0,1556.0,1921.0,1248.0,978.0,786.0,1203.0,7011.0,NaN,4704.0,4074.0,NaN,7217.0,NaN,303.0000,345.00000,231.00000,190.00000,NaN,NaT,NaT,2:37,157,1:54,114,10:22,622,NaT,NaT,NaT,NaT,NaT,NaT
1,1755.0,F,Robin Warnick,United States,North America East,CrossFit Terminus,66.0,47.0,145.0,70388.0,59325.0,109744.0,46306.0,70388.0,NaN,NaN,NaN,NaN,14769.0,20353.0,14188.0,8850.0,7446.0,8116.0,12165.0,9301.0,NaN,245.0000,265.00000,165.00000,120.00000,293.0,NaT,NaT,4:29,269,2:25,145,11:09,669,NaT,NaT,25:08,1508,1:20,80
2,1802.0,F,Melissa Kiel,United States,North America East,East Ormond Beach CrossFit,69.0,42.0,165.0,37421.0,62869.0,36822.0,23411.0,37421.0,14274.0,82750.0,17629.0,NaN,8785.0,5435.0,8117.0,7161.0,1794.0,5121.0,1126.0,1289.0,1185.0,250.0000,330.00000,200.00000,160.00000,320.0,NaT,NaT,3:30,210,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1:23,83
3,1751.0,F,Philippa Robinson,Australia,Oceania,Ocean Fitness CrossFit,68.0,41.0,216.0,35925.0,60287.0,28032.0,30128.0,35925.0,17813.0,NaN,NaN,NaN,32938.0,53678.0,8417.0,12282.0,NaN,NaN,84.0,712.0,418.0,264.5544,365.96692,213.84814,160.93726,NaN,22:54,1374,3:39,219,1:52,112,9:11,551,NaT,NaT,NaT,NaT,NaT,NaT
4,1659.0,F,Camas Barkemeyer,United States,North America West,CrossFit Kodiak Island,68.0,42.0,150.0,3944.0,11689.0,2850.0,1888.0,3944.0,NaN,1781.0,NaN,NaN,NaN,1628.0,864.0,510.0,582.0,5986.0,1042.0,2213.0,NaN,300.0000,350.00000,235.00000,190.00000,NaN,NaT,NaT,2:15,135,NaT,NaT,NaT,NaT,NaT,NaT,23:51,1431,1:00,60


In [237]:
#Output Crossfit to a CSV file
Pre_DT.to_csv('Crossfit_Women.csv', index=False)

In [238]:
# Assuming Pre_DT is your DataFrame containing the data
# Define the affiliate name
affiliate_name = "CrossFit Y'All"

# Filter out rows with the specific affiliate name
Victory = Pre_DT[Pre_DT["Affiliate Name"] == affiliate_name]

# Print or use the filtered DataFrame
print(Victory)

# Export Victory to Excel
Victory.to_excel("Victory_Data_Women.xlsx", index=False)  # This will export the DataFrame to an Excel file named "Victory_Data.xlsx" without the index column

       Athlete ID Gender  Competitor Name   Country Name         Region Name  \
14703    759524.0      F   Ali Amalfitano  United States  North America East   
21267     1011003      F    Hannah Steele  United States  North America East   
27483     1259388      F     Tammy Ahlers  United States  North America East   
56655     2079506      F  Chloe Parsemain         France              Europe   
66070     2209260      F       Eryn Wolfe  United States  North America East   
119660    2704018      F       Ana Guzman  United States  North America East   

        Affiliate Name  Height (in)  Age (Years)  Weight (lbs)  Overall Rank  \
14703   CrossFit Y'All         61.0         38.0         120.0       10299.0   
21267   CrossFit Y'All         64.0         31.0         135.0       93795.0   
27483   CrossFit Y'All          NaN         49.0           NaN       78146.0   
56655   CrossFit Y'All         63.0         28.0         160.0       38593.0   
66070   CrossFit Y'All         68.0    